In [1]:
import pandas as pd
import re
import nltk

from fuzzywuzzy import process
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

c:\Users\Matias\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matias\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Juntaremos las categorias de yelp y de google, las unificaremos y eliminaremos duplicados

In [2]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'metadata_FL_Google.json'
# Lee el archivo JSON y carga los datos en un DataFrame
df_metadada_google = pd.read_json(ruta_archivo, encoding='utf-8')


# Especifica la ruta del archivo JSON
ruta_archivo = 'yelp_business_fl.json'
# Lee el archivo JSON y carga los datos en un DataFrame
df_metadada_yelp = pd.read_json(ruta_archivo, encoding='utf-8')

In [3]:
# Muestra las primeras filas del DataFrame para verificar
df_metadada_google.head()

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,MISC,state,estado
0,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[Restaurant],5.0,1,"{'Service options': ['Dine-in', 'Delivery'], '...",None,FL
1,Fresh Point Country Buffet,"Fresh Point Country Buffet, 10525 US-19, Pinel...",0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,[Buffet restaurant],5.0,2,"{'Service options': ['Delivery'], 'Accessibili...",Permanently closed,FL
2,Oneyda's Bakery,"Oneyda's Bakery, 600 Goodlette-Frank Rd #101, ...",0x88dae191ee505917:0x6ba3e25388d3fad4,26.154754,-81.790528,"[Bakery, Deli]",4.6,19,{'Service options': ['Delivery']},Permanently closed,FL
3,Annie's Bake Shoppe,"Annie's Bake Shoppe, 10331 SW 54th St, Miami, ...",0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,25.717416,-80.361945,"[Bakery, Gift basket store]",4.3,3,"{'Service options': ['In-store shopping', 'Tak...",Open ⋅ Closes 4PM,FL
4,Tropical Park Liquors,"Tropical Park Liquors, 7971 SW 40th St Suite #...",0x88d9b99475d9fd7b:0xea6083d207b2471a,25.733914,-80.325216,[Liquor store],4.7,8,"{'Service options': ['In-store shopping'], 'Ac...",Open ⋅ Closes 12AM,FL


In [4]:
# Muestra las primeras filas del DataFrame para verificar
df_metadada_yelp.head()

,business_id,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants"
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,10588 109 Street,Edmonton,FL,53.549633,-113.508780,5.0,20,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Coffee & Tea, Cafes, Pets, Restaurants, Pet Ad..."
2,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,6880 E 82nd St,Indianapolis,FL,39.906295,-86.047463,3.5,257,"{'CoatCheck': 'False', 'Music': '{'dj': False}...","Sports Bars, American (New), American (Traditi..."
3,L_TT0BFmFwORAMaA83K54A,Village Tap Room,838 Broad Ripple Ave,Indianapolis,FL,39.869911,-86.143577,2.5,23,"{'Alcohol': 'u'none'', 'BestNights': '{'monday...","Gastropubs, Cocktail Bars, Beer Bar, Bars, Res..."
4,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,Philadelphia,FL,39.954573,-75.194894,3.0,56,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Restaurants, Automotive, Delis, Gas Stations, ..."


In [ ]:
# Cuando un local tenga varias categorias, genera un fila extra duplicando los datos pero dejando una sola categoria
df_exploded = df_metadada_google.explode('category')
# Luego, obtén los valores únicos de la columna 'category' después de la explosión
categorias_google = df_exploded['category'].unique().tolist()


# En yelp, las categorias estaban separadas por comas, las transformamos a listas.
df_metadada_yelp['categories'] = df_metadada_yelp['categories'].str.split(',')
# Cuando un local tenga varias categorias, genera un fila extra duplicando los datos pero dejando una sola categoria
df_exploded = df_metadada_yelp.explode('categories')
# Luego, obtén los valores únicos de la columna 'categories' después de la explosión
categorias_yelp = df_exploded['categories'].unique().tolist()


In [6]:
# Creamos una lista para las categorias
categorias = [] 

# A cada categoria de la lista, la transforma en minuscula y elimina posibles espacios en blanco al rincipio de la oracion
# Si la categoria no existe en la lista, la agrega
for x in categorias_google:
    if x.lower().lstrip() not in categorias:
        categorias.append(x.lower().lstrip())

# A cada categoria de la lista, la transforma en minuscula y elimina posibles espacios en blanco al rincipio de la oracion
# Si la categoria no existe en la lista, la agrega
for x in categorias_yelp:
    if x.lower().lstrip() not in categorias:
        categorias.append(x.lower().lstrip())


df_categorias = pd.DataFrame(categorias)

In [ ]:
df_categorias.head()

### Ahora tomaremos todas las categorias y unificaremos las que tengan cierta coincidencia en el nombre

In [7]:
# Limpiar y estandarizar categorías
categorias_limpias = []

for x in categorias:
    # Convertir a minúsculas y quitar caracteres especiales
    categoria_limpia = re.sub(r'[^a-zA-Z0-9\s]', '', x.lower())
    
    # Encontrar la mejor coincidencia cercana en la lista original
    resultado = process.extractOne(categoria_limpia, categorias_limpias)
    
    # Verificar si se encontró una coincidencia
    if resultado is not None:
        mejor_coincidencia, umbral = resultado
        # Si la similitud es mayor al umbral, usar la mejor coincidencia
        if umbral >= 95:
            categorias_limpias.append(mejor_coincidencia)
        else:
            categorias_limpias.append(categoria_limpia)
    else:
        categorias_limpias.append(categoria_limpia)


In [8]:
# Inicializar lematizador de WordNet
lemmatizador = WordNetLemmatizer()

#lematizar y unificar singulares y plurales
categorias_lematizadas = [lemmatizador.lemmatize(palabra) for palabra in categorias_limpias]

In [9]:
df_categorias['categorias'] = pd.DataFrame(categorias_lematizadas)
df_categorias = df_categorias.reset_index(drop=False)
df_categorias.columns= ['id_categoria', 'categoria', 'n_categoria']

In [ ]:
df_categorias

In [10]:
# Guardar el DataFrame como un archivo JSON
df_categorias.to_json('categorias_A.json', orient='records', lines=True)